In [2]:
import pandas as pd
import requests
import datetime
import time
import plotly.graph_objects as go
import openmeteo_requests
import os

import requests_cache
import pandas as pd
from retry_requests import retry
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_absolute_percentage_error, mean_squared_error

# Read locSiar file

In [3]:
locSiar = pd.read_excel('./locations2.xlsx', sheet_name='Hoja2', usecols='B:D', ).dropna()
locSiar

,Name,Location,LatLon
0,AL02 Almería,"36°50'07.5""N 2°24'08.8""W","36.835404,-2.402453"
1,AL12 Tíjola,"37°22'42.6""N 2°27'34.7""W","37.67851,-2.45965"
2,AB05 Albacete,"38°56'57.7""N 1°54'06.4""W","39.299356,-1.901778"
3,GC05 Arucas,"28°07'46.1""N 15°30'50.4""W","28.129458,-15.514004"
4,GC07 Tinajo,"29°03'00.1""N 13°39'34.5""W","29.350031,-13.659593"
5,IB04 Son Ferriol,"39°33'44.4""N 2°43'34.0""E","39.562324,2.726102"
6,IB01 Santa Eulalia,"39°00'36.2""N 1°26'23.7""E","39.010055,1.439925"
7,M03 Aranjuez,"40°02'29.9""N 3°37'49.5""W","40.041647,-3.630421"
8,V05 Cheste,"39°31'08.0""N 0°44'40.0""W","39.518889,-0.744439"
9,A19 Villena,"38°35'43.8""N 0°52'31.3""W","38.595500, -0.875361"


In [7]:
for index, row in locSiar.iterrows():
    estacion = row['Name']
    df_ = pd.read_csv('./all data siar/'+estacion+'.csv')
    df_['Fecha'] = pd.to_datetime(df_['Fecha'])
    print(estacion, df_['Fecha'].iloc[-1])

AL02 Almería 2023-06-17 00:00:00
AL12 Tíjola 2023-06-17 00:00:00
AB05 Albacete 2023-06-17 00:00:00
GC05 Arucas 2023-06-17 00:00:00
GC07 Tinajo 2023-06-17 00:00:00
IB04 Son Ferriol 2023-06-17 00:00:00
IB01 Santa Eulalia 2023-06-17 00:00:00
M03 Aranjuez 2023-06-17 00:00:00
V05 Cheste 2023-06-17 00:00:00
A19 Villena 2023-06-17 00:00:00
TE05 Teruel 2023-06-17 00:00:00
GR02 Puebla de Don Fadrique 2023-06-17 00:00:00
SA01 Ciudad Rodrigo 2023-06-17 00:00:00
BA09 Villafranca de los Barros 2023-06-17 00:00:00
CA07 Jimena de la Frontera 2023-06-17 00:00:00
SE19 IFAPA Centro Las Torres-Tomejil 2023-06-17 00:00:00
BU04 Tardajos 2023-06-17 00:00:00
LE09 Santas Martas 2023-06-17 00:00:00
C02 Boimorto 2023-06-17 00:00:00
Z04 Fabara 2023-06-17 00:00:00
HU14 Banastón 2023-06-17 00:00:00
CR03 Porzuna 2023-06-17 00:00:00
J06 Alcaudete 2023-06-17 00:00:00
Z07 Sádaba 2023-06-17 00:00:00
SG01 Gomezserracín 2023-06-17 00:00:00
GC09 Antigua - Pozo Negro 2023-06-17 00:00:00
BU07 Santa Gadea del Cid 2023-06-17 

In [10]:
import os
dirSiar = './all data siar/'
ficheros = os.listdir(dirSiar)
estacionesSiar = []
nombreEstacionesSiar = []
for f in ficheros:
    if 'csv' not in f:
        continue
    df = pd.read_csv(dirSiar+f)
    df.set_index('Fecha',inplace=True)
    df.columns = ['TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'VVMED', 'RADMED', 'ETO']
    estacionesSiar.append(df)
    nombreEstacionesSiar.append(f.split('.')[0])

In [12]:
estacionesSiar[0]

,TMAX,TMIN,HRMAX,HRMIN,VVMED,RADMED,ETO
Fecha,,,,,,,
2017-01-01,12.33,-0.39,97.4,55.71,1.20,107.523148,0.85
2017-01-02,11.72,-2.62,97.8,62.89,0.64,90.162037,0.61
2017-01-03,14.77,-2.01,98.2,38.38,1.52,76.967593,1.32
2017-01-04,14.08,5.34,88.7,59.70,2.65,115.740741,1.33
2017-01-05,15.95,1.03,95.9,38.16,1.24,118.750000,1.21
...,...,...,...,...,...,...,...
2023-06-13,28.37,14.24,95.0,35.20,1.41,246.875000,4.66
2023-06-14,27.77,11.82,96.2,34.46,1.78,307.060185,5.42
2023-06-15,30.19,13.78,95.2,31.35,1.34,305.092593,5.49


In [5]:
apiKey = '_kPYL8zGX2gs2hFNeUiKMWeKz6EfxeBDzeXO2BKqhTM2aXVJFB' # Xurui
ultimaFecha = datetime.date(2023, 6, 17)
for index, row in locSiar.iterrows():
    estacion = row['Name']
    df_ = pd.read_csv('./all data siar/'+estacion+'.csv')
    df_['Fecha'] = pd.to_datetime(df_['Fecha'])
    if df_['Fecha'].iloc[-1].date() < ultimaFecha:
        print(estacion, df_['Fecha'].iloc[-1])
        fechaInicio = df_['Fecha'].iloc[-1].date()
        fechaFin = fechaInicio + datetime.timedelta(days=91)

        df_total = pd.DataFrame()

        while fechaFin < ultimaFecha:
            print(estacion.split(' ')[0], fechaInicio, fechaFin)
            url = 'https://servicio.mapama.gob.es/apisiar/API/v1/Datos/Diarios/Estacion?Id='+estacion.split(' ')[0]+'&FechaInicial='+str(fechaInicio)+'&FechaFinal='+str(fechaFin)+'&ClaveAPI='+apiKey
            res = requests.get(url)
            df = pd.DataFrame(res.json()['Datos'])[['Fecha', 'TempMax', 'TempMin', 'HumedadMax', 'HumedadMin', 'VelViento', 'Radiacion', 'EtPMon']]
            df['Fecha'] = pd.to_datetime(df['Fecha'])
            df['Radiacion'] = df['Radiacion'].apply(lambda x: x / 0.0864)
            df_total = pd.concat([df_total, df])
            fechaInicio =  fechaFin + datetime.timedelta(days=1)
            fechaFin =  fechaFin + datetime.timedelta(days=91)
            time.sleep(60)

        print(estacion.split(' ')[0], fechaInicio, ultimaFecha)
        url = 'https://servicio.mapama.gob.es/apisiar/API/v1/Datos/Diarios/Estacion?Id='+estacion.split(' ')[0]+'&FechaInicial='+str(fechaInicio)+'&FechaFinal='+str(ultimaFecha)+'&ClaveAPI='+apiKey
        res = requests.get(url)
        df = pd.DataFrame(res.json()['Datos'])[['Fecha', 'TempMax', 'TempMin', 'HumedadMax', 'HumedadMin', 'VelViento', 'Radiacion', 'EtPMon']]
        df['Fecha'] = pd.to_datetime(df['Fecha'])
        df['Radiacion'] = df['Radiacion'].apply(lambda x: x / 0.0864)
        df_total = pd.concat([df_total, df])

        display(df_total)
        df_total.to_csv('./all data siar/'+estacion+'.csv', mode='a', header=False, index=False)
        time.sleep(60)

SA01 Ciudad Rodrigo 2023-01-01 00:00:00
SA01 2023-01-01 2023-04-02
SA01 2023-04-03 2023-06-17


,Fecha,TempMax,TempMin,HumedadMax,HumedadMin,VelViento,Radiacion,EtPMon
0,2023-01-01,14.910,9.380,97.5,46.13,3.282,78.865741,1.621675
1,2023-01-02,10.420,-0.562,100.0,60.26,0.525,63.807870,0.518022
2,2023-01-03,10.880,-2.543,100.0,60.39,0.515,77.881944,0.486417
3,2023-01-04,2.793,-1.552,100.0,100.00,0.542,34.710648,0.321417
4,2023-01-05,3.058,-1.619,100.0,100.00,0.575,37.106481,0.316727
...,...,...,...,...,...,...,...,...
71,2023-06-13,23.920,13.610,97.6,41.56,1.024,211.458333,3.620395
72,2023-06-14,26.550,13.670,95.3,38.39,0.747,296.527778,4.665490
73,2023-06-15,30.100,12.750,97.9,28.36,0.549,341.666667,5.303614
74,2023-06-16,31.810,13.280,96.4,28.95,0.694,331.712963,5.485945


BA09 Villafranca de los Barros 2023-01-01 00:00:00
BA09 2023-01-01 2023-04-02
BA09 2023-04-03 2023-06-17


,Fecha,TempMax,TempMin,HumedadMax,HumedadMin,VelViento,Radiacion,EtPMon
0,2023-01-01,20.37,10.950,100.00,31.52,3.444,74.861111,2.643290
1,2023-01-02,12.67,4.711,100.00,67.83,0.955,84.143519,0.715546
2,2023-01-03,11.61,2.260,100.00,71.90,0.829,110.648148,0.626053
3,2023-01-04,14.65,-0.067,100.00,84.40,0.825,106.597222,0.608029
4,2023-01-05,16.18,3.517,100.00,67.64,0.941,108.449074,0.811737
...,...,...,...,...,...,...,...,...
71,2023-06-13,26.06,16.440,86.60,34.62,2.457,297.916667,5.630192
72,2023-06-14,30.57,14.150,85.80,28.93,1.280,328.935185,5.831248
73,2023-06-15,34.29,18.380,81.40,21.99,0.956,350.231481,6.327041
74,2023-06-16,37.01,19.630,56.24,20.00,1.282,316.203704,6.624869


CA07 Jimena de la Frontera 2023-01-01 00:00:00
CA07 2023-01-01 2023-04-02
CA07 2023-04-03 2023-06-17


...

,Fecha,TempMax,TempMin,HumedadMax,HumedadMin,VelViento,Radiacion,EtPMon
0,2022-12-31,13.730,-3.092,98.2,27.57,3.089,98.726852,1.869722
1,2023-01-01,13.200,7.240,82.6,38.68,4.720,67.233796,2.274664
2,2023-01-02,8.380,-2.755,98.3,74.40,1.338,51.875000,0.542732
3,2023-01-03,6.948,-5.376,99.5,64.23,0.630,93.171296,0.381022
4,2023-01-04,8.090,-7.660,98.5,58.71,0.545,105.208333,0.379948
...,...,...,...,...,...,...,...,...
72,2023-06-13,23.130,10.330,99.9,45.45,1.659,257.638889,4.145428
73,2023-06-14,25.400,7.850,99.5,39.52,1.143,246.990741,4.089995
74,2023-06-15,28.950,8.660,99.0,27.28,0.817,356.018519,5.479672
75,2023-06-16,31.170,9.530,98.7,24.26,0.975,344.675926,5.739444


GC09 Antigua - Pozo Negro 2023-01-01 00:00:00
GC09 2023-01-01 2023-04-02
GC09 2023-04-03 2023-06-17


,Fecha,TempMax,TempMin,HumedadMax,HumedadMin,VelViento,Radiacion,EtPMon
0,2023-01-01,23.19,14.92,79.80,39.56,1.351,167.245370,2.475995
1,2023-01-02,22.38,12.12,69.34,26.65,0.868,172.800926,2.147211
2,2023-01-03,22.33,15.79,80.60,53.22,3.501,161.226852,3.136289
3,2023-01-04,22.59,17.99,83.80,42.84,3.775,149.074074,3.545359
4,2023-01-05,22.18,14.99,84.00,39.83,2.383,159.490741,2.939340
...,...,...,...,...,...,...,...,...
71,2023-06-13,26.80,20.26,84.00,47.72,3.026,297.106481,5.714409
72,2023-06-14,27.14,20.26,79.40,53.21,3.445,328.935185,6.124252
73,2023-06-15,28.67,19.46,80.20,48.19,2.989,335.069444,6.353686
74,2023-06-16,30.54,19.73,81.60,44.84,2.934,337.731481,6.713211


BU07 Santa Gadea del Cid 2023-01-01 00:00:00
BU07 2023-01-01 2023-04-02
BU07 2023-04-03 2023-06-17


,Fecha,TempMax,TempMin,HumedadMax,HumedadMin,VelViento,Radiacion,EtPMon
0,2023-01-01,19.48,10.080,69.39,32.10,2.769,40.358796,2.558734
1,2023-01-02,12.58,2.881,98.40,61.01,1.267,51.967593,0.707778
2,2023-01-03,10.97,-1.433,100.00,59.84,0.595,62.349537,0.459297
3,2023-01-04,11.84,-2.859,100.00,55.84,0.561,75.115741,0.449470
4,2023-01-05,8.69,-2.519,100.00,77.30,0.496,57.511574,0.344367
...,...,...,...,...,...,...,...,...
71,2023-06-13,21.93,10.480,100.00,60.65,1.024,166.666667,2.750226
72,2023-06-14,23.75,11.700,100.00,57.39,1.542,260.879630,4.071734
73,2023-06-15,27.76,11.300,100.00,40.53,1.051,293.981481,4.761431
74,2023-06-16,32.18,10.150,100.00,26.26,0.788,278.703704,4.891780


# Get Real SIAR data

In [16]:
apiKey = '_kPYL8zGX2gs2hFNeUiKMWeKz6EfxeBDzeXO2BKqhTM2aXVJFB' # Xurui
#apiKey = '_KspYoVmwjXWh1FjnKp2OfMRyKAMqaxZDUW8AY2wStnutoXmCM' # Yu
ultimaFecha = datetime.date(2024,6,25)
for index, row in locSiar.iterrows():
    estacion = row['Name']
    fechaInicio = datetime.date(2023,6,18)
    fechaFin = fechaInicio + datetime.timedelta(days=91)

    df_total = pd.DataFrame()

    while fechaFin < ultimaFecha:
        print(estacion.split(' ')[0], fechaInicio, fechaFin)
        url = 'https://servicio.mapama.gob.es/apisiar/API/v1/Datos/Diarios/Estacion?Id='+estacion.split(' ')[0]+'&FechaInicial='+str(fechaInicio)+'&FechaFinal='+str(fechaFin)+'&ClaveAPI='+apiKey
        res = requests.get(url)
        df = pd.DataFrame(res.json()['Datos'])[['Fecha', 'TempMax', 'TempMin', 'HumedadMax', 'HumedadMin', 'VelViento', 'Radiacion', 'EtPMon']]
        df['Fecha'] = pd.to_datetime(df['Fecha'])
        #df['Radiacion'] = df['Radiacion'].apply(lambda x: x / 0.0864)
        df_total = pd.concat([df_total, df])
        fechaInicio =  fechaFin + datetime.timedelta(days=1)
        fechaFin =  fechaFin + datetime.timedelta(days=91)
        time.sleep(60)

    print(estacion.split(' ')[0], fechaInicio, ultimaFecha)
    url = 'https://servicio.mapama.gob.es/apisiar/API/v1/Datos/Diarios/Estacion?Id='+estacion.split(' ')[0]+'&FechaInicial='+str(fechaInicio)+'&FechaFinal='+str(ultimaFecha)+'&ClaveAPI='+apiKey
    res = requests.get(url)
    df = pd.DataFrame(res.json()['Datos'])[['Fecha', 'TempMax', 'TempMin', 'HumedadMax', 'HumedadMin', 'VelViento', 'Radiacion', 'EtPMon']]
    df['Fecha'] = pd.to_datetime(df['Fecha'])
    #df['Radiacion'] = df['Radiacion'].apply(lambda x: x / 0.0864)
    df_total = pd.concat([df_total, df])

    df_total.to_csv('./siarRealDataForForecast/'+estacion+'.csv', index=False)
    time.sleep(60)


C02 2023-06-18 2023-09-17
C02 2023-09-18 2023-12-17
C02 2023-12-18 2024-03-17
C02 2024-03-18 2024-06-16
C02 2024-06-17 2024-06-25
Z04 2023-06-18 2023-09-17
Z04 2023-09-18 2023-12-17
Z04 2023-12-18 2024-03-17
Z04 2024-03-18 2024-06-16
Z04 2024-06-17 2024-06-25
HU14 2023-06-18 2023-09-17
HU14 2023-09-18 2023-12-17
HU14 2023-12-18 2024-03-17
HU14 2024-03-18 2024-06-16
HU14 2024-06-17 2024-06-25
CR03 2023-06-18 2023-09-17
CR03 2023-09-18 2023-12-17
CR03 2023-12-18 2024-03-17
CR03 2024-03-18 2024-06-16
CR03 2024-06-17 2024-06-25
J06 2023-06-18 2023-09-17
J06 2023-09-18 2023-12-17
J06 2023-12-18 2024-03-17
J06 2024-03-18 2024-06-16
J06 2024-06-17 2024-06-25
Z07 2023-06-18 2023-09-17
Z07 2023-09-18 2023-12-17
Z07 2023-12-18 2024-03-17
Z07 2024-03-18 2024-06-16
Z07 2024-06-17 2024-06-25
SG01 2023-06-18 2023-09-17
SG01 2023-09-18 2023-12-17
SG01 2023-12-18 2024-03-17
SG01 2024-03-18 2024-06-16
SG01 2024-06-17 2024-06-25
GC09 2023-06-18 2023-09-17
GC09 2023-09-18 2023-12-17
GC09 2023-12-18 2024-

In [81]:
s = pd.read_csv('./siarRealDataForForecast/AL02 Almería.csv')
s['Radiacion'] = s['Radiacion'].apply(lambda x: x / 0.0864)
fig = go.Figure()
for c in s.columns[1:]:
    fig.add_trace(go.Scatter(x=s['Fecha'], y=s[c],
                                name=c, mode='lines'))
fig.show()

In [45]:

url = 'https://servicio.mapama.gob.es/apisiar/API/v1/Datos/Diarios/Estacion?Id=A19&FechaInicial=2023-06-15&FechaFinal=2023-09-18&ClaveAPI=_KspYoVmwjXWh1FjnKp2OfMRyKAMqaxZDUW8AY2wStnutoXmCM'
res = requests.get(url)
res.json()

{'Datos': [{'Fecha': '2023-06-15T00:00:00',
   'TempMedia': 21.73,
   'TempMax': 30.19,
   'HorMinTempMax': 1310,
   'TempMin': 13.78,
   'HorMinTempMin': 330,
   'HumedadMedia': 65.17,
   'HumedadMax': 95.2,
   'HorMinHumMax': 420,
   'HumedadMin': 31.35,
   'HorMinHumMin': 1230,
   'VelViento': 1.341,
   'DirViento': 140.7,
   'VelVientoMax': 5.067,
   'HorMinVelMax': 1414,
   'DirVientoVelMax': 201.5,
   'Radiacion': 26.36,
   'Precipitacion': 0.0,
   'TempSuelo1': None,
   'TempSuelo2': None,
   'EtPMon': 5.489788,
   'PePMon': 0.0,
   'Estacion': 'A19'},
  {'Fecha': '2023-06-16T00:00:00',
   'TempMedia': 22.44,
   'TempMax': 30.19,
   'HorMinTempMax': 1220,
   'TempMin': 12.96,
   'HorMinTempMin': 310,
   'HumedadMedia': 62.42,
   'HumedadMax': 94.2,
   'HorMinHumMax': 510,
   'HumedadMin': 35.87,
   'HorMinHumMin': 1000,
   'VelViento': 1.463,
   'DirViento': 169.6,
   'VelVientoMax': 7.91,
   'HorMinVelMax': 1117,
   'DirVientoVelMax': 213.3,
   'Radiacion': 27.32,
   'Precipita

In [47]:
df = pd.DataFrame(res.json()['Datos'])[['Fecha', 'TempMax', 'TempMin', 'HumedadMax', 'HumedadMin', 'VelViento', 'Radiacion', 'EtPMon']]
df['Fecha'] = pd.to_datetime(df['Fecha'])
df['EtPMon'] = round(df['EtPMon'],2)
df['VelViento'] = round(df['VelViento'], 2 )
df

,Fecha,TempMax,TempMin,HumedadMax,HumedadMin,VelViento,Radiacion,EtPMon
0,2023-06-15,30.19,13.78,95.2,31.35,1.34,26.36,5.49
1,2023-06-16,30.19,12.96,94.2,35.87,1.46,27.32,5.64
2,2023-06-17,31.27,14.05,92.5,26.01,1.68,28.22,6.19
3,2023-06-18,27.44,15.79,90.0,35.97,1.56,11.41,3.49
4,2023-06-19,28.86,16.46,93.8,32.31,3.30,25.55,6.35
...,...,...,...,...,...,...,...,...
91,2023-09-14,28.80,18.49,93.1,40.89,1.72,15.13,3.73
92,2023-09-15,28.97,17.07,96.3,47.96,1.27,15.96,3.40
93,2023-09-16,26.52,14.99,98.4,60.71,0.89,14.03,2.68
94,2023-09-17,28.59,18.35,97.7,52.98,1.45,18.07,3.63


In [14]:
url = 'https://servicio.mapama.gob.es/apisiar/API/v1/Info/Accesos?ClaveAPI=_KspYoVmwjXWh1FjnKp2OfMRyKAMqaxZDUW8AY2wStnutoXmCM'

usage = requests.get(url)
usage.json()

{'Datos': [{'NumAccesosMinutoActual': 1,
   'MaxAccesosMinuto': 15,
   'NumAccesosDiaActual': 143,
   'MaxAccesosDia': 240,
   'RegistrosAcumuladosMinuto': 1,
   'MaxRegistrosMinuto': 100,
   'RegistrosAcumuladosDia': 9091,
   'MaxRegistrosDia': 10000}],
 'MensajeRespuesta': None}

In [9]:
url = 'https://servicio.mapama.gob.es/apisiar/API/v1/Info/Estaciones?ClaveAPI=-jSvrV9ft-0VPnsOUL2mnB09K5Py69xAUbxLrzscQ70qjuYR9e'
res = requests.get(url)
res.json()

{'Datos': [{'Estacion': 'Villena',
   'Codigo': 'A01',
   'Termino': 'Villena',
   'Longitud': '005304000W',
   'Latitud': '384035000N',
   'Altitud': 519,
   'XUTM': 684017.0,
   'YUTM': 4283000.0,
   'Huso': 30,
   'Fecha_Instalacion': '1999-11-10T00:00:00',
   'Fecha_Baja': '2000-03-20T00:00:00'},
  {'Estacion': 'Camp de Mirra',
   'Codigo': 'A02',
   'Termino': 'Campo de Mirra/Camp de Mirra, el',
   'Longitud': '004622000W',
   'Latitud': '384045000N',
   'Altitud': 589,
   'XUTM': 693720.0,
   'YUTM': 4283550.0,
   'Huso': 30,
   'Fecha_Instalacion': '1999-11-10T00:00:00',
   'Fecha_Baja': None},
  {'Estacion': 'Vila Joiosa',
   'Codigo': 'A03',
   'Termino': 'Villajoyosa/Vila Joiosa, la',
   'Longitud': '001522000W',
   'Latitud': '383140000N',
   'Altitud': 73,
   'XUTM': 739183.0,
   'YUTM': 4267960.0,
   'Huso': 30,
   'Fecha_Instalacion': '1999-11-11T00:00:00',
   'Fecha_Baja': None},
  {'Estacion': 'Ondara',
   'Codigo': 'A04',
   'Termino': 'Ondara',
   'Longitud': '0000230

# Get Prediction SIAR data

## Hourly (without ETo)

In [52]:
import time

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

for index, row in locSiar.iterrows():
    print(row['Name'], row['LatLon'])

    url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
    params = {
        "latitude": float(row['LatLon'].split(',')[0]),
        "longitude": float(row['LatLon'].split(',')[1]),
        "start_date": "2023-06-17",
        "end_date": "2024-06-26",
        "hourly": ["temperature_2m", "relative_humidity_2m", "wind_speed_10m", "shortwave_radiation"],
        "wind_speed_unit": "ms",
        "models": "gfs_global"
    }
    responses = openmeteo.weather_api(url, params=params)

    # Process first location. Add a for-loop for multiple locations or weather models
    response = responses[0]
    print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
    print(f"Elevation {response.Elevation()} m asl")
    print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
    print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

    # Process hourly data. The order of variables needs to be the same as requested.
    hourly = response.Hourly()
    hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
    hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
    hourly_wind_speed_10m = hourly.Variables(2).ValuesAsNumpy()
    hourly_shortwave_radiation = hourly.Variables(3).ValuesAsNumpy()

    hourly_data = {"date": pd.date_range(
        start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
        end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
        freq = pd.Timedelta(seconds = hourly.Interval()),
        inclusive = "left"
    )}
    hourly_data["temperature_2m"] = hourly_temperature_2m
    hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
    hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
    hourly_data["shortwave_radiation"] = hourly_shortwave_radiation


    hourly_dataframe_global = pd.DataFrame(data = hourly_data)
    print(hourly_dataframe_global)
    hourly_dataframe_global['date'] = hourly_dataframe_global['date'].apply(lambda x: x.replace(tzinfo=None))
    hourly_dataframe_global.columns=['dates', 'temp', 'rh', 'wind', 'solar_rad']
    hourly_dataframe_global.to_csv('./forecastSiar/'+row['Name']+'.csv', index=False)
    time.sleep(1)

AL02 Almería 36.835404,-2.402453
Coordinates 36.843475341796875°N -2.34375°E
Elevation 6.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
                          date  temperature_2m  relative_humidity_2m   
0    2023-06-17 00:00:00+00:00       26.940498                  41.0  \
1    2023-06-17 01:00:00+00:00       26.840500                  41.0   
2    2023-06-17 02:00:00+00:00       26.890499                  41.0   
3    2023-06-17 03:00:00+00:00       26.940498                  41.0   
4    2023-06-17 04:00:00+00:00       26.340500                  42.0   
...                        ...             ...                   ...   
9019 2024-06-26 19:00:00+00:00       25.440498                  57.0   
9020 2024-06-26 20:00:00+00:00       24.640499                  61.0   
9021 2024-06-26 21:00:00+00:00       23.440498                  66.0   
9022 2024-06-26 22:00:00+00:00       22.840500                  69.0   
9023 2024-06-26 23:00:00+00:00       23.040499             

## Diaria (Con ETo)

In [32]:
import openmeteo_requests

import requests_cache
import pandas as pd
from retry_requests import retry

# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)
openmeteo = openmeteo_requests.Client(session = retry_session)

for index, row in locSiar.iterrows():
	print(row['Name'], row['LatLon'])

	url = "https://historical-forecast-api.open-meteo.com/v1/forecast"
	params = {
		"latitude": float(row['LatLon'].split(',')[0]),
        "longitude": float(row['LatLon'].split(',')[1]),
		"start_date": "2023-06-17",
        "end_date": "2024-06-26",
		"daily": "et0_fao_evapotranspiration",
		"models": "gfs_global"
	}
	responses = openmeteo.weather_api(url, params=params)
    
	# Process first location. Add a for-loop for multiple locations or weather models
	response = responses[0]
	print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
	print(f"Elevation {response.Elevation()} m asl")
	print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
	print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

	# Process daily data. The order of variables needs to be the same as requested.
	daily = response.Daily()
	daily_et0_fao_evapotranspiration = daily.Variables(0).ValuesAsNumpy()

	daily_data = {"date": pd.date_range(
		start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
		end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
		freq = pd.Timedelta(seconds = daily.Interval()),
		inclusive = "left"
	)}
	daily_data["et0_fao_evapotranspiration"] = daily_et0_fao_evapotranspiration

	daily_dataframe = pd.DataFrame(data = daily_data)
	daily_dataframe['date'] = daily_dataframe['date'].apply(lambda x: datetime.datetime.strptime(str(x.replace(tzinfo=None)), '%Y-%m-%d %H:%M:%S').date() )
	print(daily_dataframe)
	daily_dataframe.to_csv('./forecastSiarDaily/'+row['Name']+'.csv', index=False)
	time.sleep(1)



AL02 Almería 36.835404,-2.402453
Coordinates 36.843475341796875°N -2.34375°E
Elevation 6.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
           date  et0_fao_evapotranspiration
0    2023-06-17                    9.257853
1    2023-06-18                    6.505656
2    2023-06-19                    6.006471
3    2023-06-20                    5.896463
4    2023-06-21                    6.046155
..          ...                         ...
371  2024-06-22                    6.292175
372  2024-06-23                    6.717399
373  2024-06-24                    7.543763
374  2024-06-25                    7.373966
375  2024-06-26                    6.058483

[376 rows x 2 columns]
AL12 Tíjola 37.67851,-2.45965
Coordinates 37.66352081298828°N -2.34375°E
Elevation 1379.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s
           date  et0_fao_evapotranspiration
0    2023-06-17                    6.635606
1    2023-06-18                    4.561783
2    2023-06-19    

# Check errors between real and forecast

## Variables errors

In [6]:
def hourlyToDaily(df):
    punto = [l[1] for l in list(df.groupby([df['dates'].dt.date]))]
    FECHA = []
    TMAX = []
    TMIN = []
    HRMAX = []
    HRMIN = []
    VVMED = []
    RADMED = []
    for p in punto:
        FECHA.append(pd.to_datetime(p['dates']).dt.date.iloc[0])
        TMAX.append(p['temp'].max())
        TMIN.append(p['temp'].min())
        HRMAX.append(p['rh'].max())
        HRMIN.append(p['rh'].min())
        VVMED.append(p['wind'].mean())
        RADMED.append(p['solar_rad'].mean())

    return pd.DataFrame({
        "FECHA": pd.to_datetime(FECHA),
        "TMAX": TMAX,
        "TMIN": TMIN,
        "HRMAX": HRMAX,
        "HRMIN": HRMIN,
        "VVMED": VVMED,
        "RADMED": RADMED
    })

def medidasError(y_test, y_pred, txt):
    medidas =[]
    medidas.append(txt)
    medidas.append(np.corrcoef(y_test, y_pred)[0][1]**2)
    medidas.append(mean_absolute_error(y_true=y_test,y_pred=y_pred))
    medidas.append(mean_absolute_percentage_error(y_true=y_test,y_pred=y_pred)*100)
    medidas.append(mean_squared_error(y_true=y_test,y_pred=y_pred,squared=False)/np.mean(y_test)*100)
    medidas.append(mean_squared_error(y_true=y_test,y_pred=y_pred,squared=False))
    return medidas

def compareHourlyDaily(hourly, dfDaily, weatherModel):
    hourly.columns = ['dates', 'temp', 'rh', 'wind', 'solar_rad']
    hourly['dates'] = pd.to_datetime(hourly['dates'])
    hourly = hourlyToDaily(hourly).add_suffix('-'+weatherModel)
    hourly['VVMED-'+weatherModel] = hourly['VVMED-'+weatherModel].apply(lambda x: x*4.87/np.log(67.8*10-5.42))
    hourly = hourly.rename(index=str, columns={'FECHA-'+weatherModel:'FECHA'})
    df_join = pd.merge(dfDaily.reset_index(), hourly, how='inner', on='FECHA')
    
    fig = go.Figure()
    all_medidas = []
    for c in df_join.columns[1:]:
        if 'VVMED' not in c:
            continue
        var = c.split('-')[0]
        if (var != 'ETO') & (weatherModel not in c):
            all_medidas.append(medidasError(df_join[var], df_join[c+'-'+weatherModel], var))
        fig.add_trace(go.Scatter(x=df_join['FECHA'], y=df_join[c],
                                name=c, mode='lines'))
    fig.show()
    return pd.DataFrame(all_medidas, columns=['Param', 'R2', 'MAE', 'MAPE', 'CVRMSE', 'RMSE'])
    #return df_join

### WB

In [9]:

def convertirComa(x):
    if type(x) == str:
        return x.replace(",", ".")
    else:
        return x
def leerEstacionDatos(path):
    estacionDatas = pd.read_csv(path, encoding='ISO-8859-1', sep=";")
    estacionDatas.columns = ['ESTACION', 'MUNICIPIO', 'PARAJE', 'HORAS', 'FECHA', 'ETO','TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'RADMED','VVMED', '-']
    estacionDatas = estacionDatas.drop(columns=['ESTACION', 'MUNICIPIO', 'PARAJE', 'HORAS', '-'])
    estacionDatas = estacionDatas.reset_index().drop(columns='index')
    estacionDatas['FECHA'] = pd.to_datetime(estacionDatas['FECHA'], format="%d/%m/%y")
    estacionDatas.index = estacionDatas['FECHA']
    estacionDatas.drop(columns='FECHA', inplace=True)
    estacionDatas.dropna(inplace=True)
    for i in estacionDatas.columns:
        estacionDatas[i] = pd.to_numeric(estacionDatas[i].apply(lambda x : convertirComa(x)))
    return estacionDatas

all_errors = pd.DataFrame()
dir = './forecastTest/'
for station in os.listdir(dir):

    print(station)
    pred = pd.read_csv(dir+station+'/WB-'+station+'.csv')[['dates', 'temp', 'rh', 'wind', 'solar_rad']]

    real = leerEstacionDatos('./all data murcia/' + station.split('-')[0] + '.csv') 
    errors = compareHourlyDaily(pred, real, 'WB')
    errors['Station'] = station
    all_errors = pd.concat([all_errors, errors])
all_errors

AL41-alhama


CA73-cartagena


CA91-fuenteAlamo


CI42-cieza


CR12-caravaca


JU71-jumilla


LO51-aguilas


MO22-molinaSegura


,Param,R2,MAE,MAPE,CVRMSE,RMSE,Station
0,VVMED,0.450793,0.598271,3.887135e+15,43.282332,0.741318,AL41-alhama
0,VVMED,0.543941,1.020875,7.927693e+01,93.675472,1.279945,CA73-cartagena
0,VVMED,0.473187,0.957019,1.162328e+02,127.844174,1.153725,CA91-fuenteAlamo
0,VVMED,0.537692,0.797841,9.984149e+01,100.597935,0.971690,CI42-cieza
0,VVMED,0.749907,0.529709,3.630040e+01,41.714711,0.680084,CR12-caravaca
0,VVMED,0.584162,0.418975,2.099308e+01,29.910527,0.583746,JU71-jumilla
0,VVMED,0.772761,0.332846,1.568654e+01,22.826998,0.465039,LO51-aguilas
0,VVMED,0.625577,0.492941,3.276227e+01,36.183795,0.607849,MO22-molinaSegura


In [93]:
all_errors.groupby(['Param']).describe()[[(    'R2',  'mean'),
            (   'MAE',  'mean'),
            (  'MAPE',  'mean'),
            (  'CVRMSE',  'mean'),
            (  'RMSE',  'mean'),
            ]].iloc[::-1]

,R2,MAE,MAPE,CVRMSE,RMSE
,mean,mean,mean,mean,mean
Param,,,,,
VVMED,0.592253,0.643560,4.858919e+14,62.004493,0.810424
TMIN,0.940227,1.474561,4.010686e+01,15.305685,1.806292
TMAX,0.972898,0.932535,4.032919e+00,4.823410,1.176840
RADMED,0.894179,20.424941,1.315937e+01,14.397379,30.632263
HRMIN,0.773444,5.335667,1.742207e+01,19.970850,6.862483
HRMAX,0.557683,7.148452,9.539824e+00,11.580677,9.235299


In [94]:
import plotly.express as px
for metric in ['MAE', 'CVRMSE']:
    fig = px.box(all_errors, x="Param", y=metric)
    fig.update_traces(boxmean=True)
    fig.show()

### SIAR

In [87]:
all_errors = pd.DataFrame()
for index, row in locSiar.iterrows():
    stationCode = row['Name']
    print(stationCode)

    siarPred = pd.read_csv('./forecastSiar/'+stationCode+'.csv')
    
    siarReal = pd.read_csv('./siarRealDataForForecast/'+stationCode+'.csv')
    siarReal['Radiacion'] = siarReal['Radiacion'].apply(lambda x: x / 0.0864)
    siarReal.columns = ['FECHA', 'TMAX', 'TMIN', 'HRMAX', 'HRMIN', 'VVMED', 'RADMED', 'ETO']
    siarReal['FECHA'] = pd.to_datetime(siarReal['FECHA'])
    siarReal.set_index('FECHA', inplace=True)
    siarReal.dropna(inplace=True)

    errors = compareHourlyDaily(siarPred, siarReal, 'NOAA')
    errors['Station'] = stationCode
    all_errors = pd.concat([all_errors, errors])
all_errors

AL02 Almería
AL12 Tíjola
AB05 Albacete
GC05 Arucas
GC07 Tinajo
IB04 Son Ferriol
IB01 Santa Eulalia
M03 Aranjuez
V05 Cheste
A19 Villena
TE05 Teruel
GR02 Puebla de Don Fadrique
SA01 Ciudad Rodrigo
BA09 Villafranca de los Barros
CA07 Jimena de la Frontera
SE19 IFAPA Centro Las Torres-Tomejil
BU04 Tardajos
LE09 Santas Martas
C02 Boimorto
Z04 Fabara
HU14 Banastón
CR03 Porzuna
J06 Alcaudete
Z07 Sádaba
SG01 Gomezserracín
GC09 Antigua - Pozo Negro
BU07 Santa Gadea del Cid


,Param,R2,MAE,MAPE,CVRMSE,RMSE,Station
0,TMAX,0.940661,1.391798,5.786879,6.669603,1.661759,AL02 Almería
1,TMIN,0.869094,3.253075,27.562137,24.651003,3.788565,AL02 Almería
2,HRMAX,0.328689,14.938486,17.158455,20.192777,17.521862,AL02 Almería
3,HRMIN,0.549385,7.192784,18.729768,21.856666,9.316675,AL02 Almería
4,VVMED,0.716159,1.566222,146.415696,174.014879,1.908699,AL02 Almería
...,...,...,...,...,...,...,...
1,TMIN,0.785947,2.562131,138.922516,44.489977,3.267174,BU07 Santa Gadea del Cid
2,HRMAX,0.144217,8.902674,9.362986,12.876762,12.223697,BU07 Santa Gadea del Cid
3,HRMIN,0.630438,7.595134,17.296169,22.613456,10.379129,BU07 Santa Gadea del Cid
4,VVMED,0.851738,0.759471,45.597444,54.317033,0.999727,BU07 Santa Gadea del Cid


In [89]:
all_errors.groupby(['Param']).describe()[[(    'R2',  'mean'),
            (   'MAE',  'mean'),
            (  'MAPE',  'mean'),
            (  'CVRMSE',  'mean'),
            (  'RMSE',  'mean'),
            ]].iloc[::-1]

,R2,MAE,MAPE,CVRMSE,RMSE
,mean,mean,mean,mean,mean
Param,,,,,
VVMED,0.712178,1.250889,104.721502,111.155015,1.503700
TMIN,0.818547,2.875941,209.625135,39.613446,3.449563
TMAX,0.935272,1.680696,17.768378,9.006990,2.060534
RADMED,0.891205,23.609483,17.281069,17.638894,34.149651
HRMIN,0.715990,7.813011,42.352246,25.453565,10.531796
HRMAX,0.334697,12.526090,23.947292,17.585974,15.848692


In [56]:
all_errors.groupby(['Param']).describe()[[(    'R2',  'mean'),
            (   'MAE',  'mean'),
            (  'MAPE',  'mean'),
            (  'CVRMSE',  'mean'),
            (  'RMSE',  'mean'),
            ]].iloc[::-1]

,R2,MAE,MAPE,CVRMSE,RMSE
,mean,mean,mean,mean,mean
Param,,,,,
VVMED,0.712178,1.250889,104.721502,111.155015,1.503700
TMIN,0.818547,2.875941,209.625135,39.613446,3.449563
TMAX,0.935272,1.680696,17.768378,9.006990,2.060534
RADMED,0.891205,23.609483,17.281069,17.638894,34.149651
HRMIN,0.715990,7.813011,42.352246,25.453565,10.531796
HRMAX,0.334697,12.526090,23.947292,17.585974,15.848692


In [91]:
import plotly.express as px
for metric in ['MAE', 'CVRMSE']:
    fig = px.box(all_errors, x="Param", y=metric, points='all')
    fig.update_traces(boxmean=True)
    fig.show()

## ETo Errors

In [35]:
def medidasError(y_test, y_pred,stationCode):
    medidas =[]
    medidas.append(stationCode)
    medidas.append(np.corrcoef(y_test, y_pred)[0][1]**2)
    medidas.append(mean_absolute_error(y_true=y_test,y_pred=y_pred))
    medidas.append(mean_absolute_percentage_error(y_true=y_test,y_pred=y_pred)*100)
    medidas.append(mean_squared_error(y_true=y_test,y_pred=y_pred,squared=False)/np.mean(y_test)*100)
    medidas.append(mean_squared_error(y_true=y_test,y_pred=y_pred,squared=False))
    return medidas

all_errors = []
for index, row in locSiar.iterrows():
    stationCode = row['Name']
    print(stationCode)

    siarPred = pd.read_csv('./forecastSiarDaily/'+stationCode+'.csv')
    siarPred.columns = ['Fecha', 'ET0Noaa']
    siarReal = pd.read_csv('./siarRealDataForForecast/'+stationCode+'.csv')[['Fecha', 'EtPMon']]
    df_join = pd.merge(siarPred, siarReal, how='inner', on='Fecha')
    df_join.dropna(inplace=True)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=df_join['Fecha'], y=df_join['ET0Noaa'],name='ET0Noaa', mode='lines'))
    fig.add_trace(go.Scatter(x=df_join['Fecha'], y=df_join['EtPMon'],name='EtPMon', mode='lines'))
    fig.show()
    errores = medidasError(y_test=df_join['EtPMon'], y_pred=df_join['ET0Noaa'], stationCode=stationCode)
    print('Station','R2', 'MAE', 'MAPE', 'CVRMSE', 'RMSE')
    print(errores)
    all_errors.append(errores)
    
all_errors = pd.DataFrame(all_errors, columns=['Station','R2', 'MAE', 'MAPE', 'CVRMSE', 'RMSE'])
all_errors

AL02 Almería


Station R2 MAE MAPE CVRMSE RMSE
['AL02 Almería', 0.9073755139021719, 0.9737478077027028, 37.20756741731107, 32.54326862501304, 1.1474162174187412]
AL12 Tíjola


Station R2 MAE MAPE CVRMSE RMSE
['AL12 Tíjola', 0.8686204046617754, 0.7558609639037434, 23.463573528998964, 29.342208416660338, 1.006925512852021]
AB05 Albacete


Station R2 MAE MAPE CVRMSE RMSE
['AB05 Albacete', 0.9150412526150988, 0.845700037433155, 33.4547151394409, 27.25854446976257, 1.0370808007508867]
GC05 Arucas


Station R2 MAE MAPE CVRMSE RMSE
['GC05 Arucas', 0.6622776935089061, 0.5336496050802139, 17.16006999113658, 18.862296956357806, 0.6667132461294228]
GC07 Tinajo


...

Station R2 MAE MAPE CVRMSE RMSE
['Z07 Sádaba', 0.9568454475977288, 0.7027697600802139, 33.849189448243955, 25.590481828143997, 0.8230279108696805]
SG01 Gomezserracín


Station R2 MAE MAPE CVRMSE RMSE
['SG01 Gomezserracín', 0.9410385334919042, 0.7691582837165776, 41.314780195393254, 31.613422239237615, 0.9534753852264721]
GC09 Antigua - Pozo Negro


Station R2 MAE MAPE CVRMSE RMSE
['GC09 Antigua - Pozo Negro', 0.8342381103424418, 0.532416098395722, 11.527883230644093, 13.83099017665855, 0.709676124240527]
BU07 Santa Gadea del Cid


Station R2 MAE MAPE CVRMSE RMSE
['BU07 Santa Gadea del Cid', 0.9097663197898933, 0.6205181048930482, 31.81104448917875, 27.404444142409496, 0.7657462999263328]


,Station,R2,MAE,MAPE,CVRMSE,RMSE
0,AL02 Almería,0.907376,0.973748,37.207567,32.543269,1.147416
1,AL12 Tíjola,0.868620,0.755861,23.463574,29.342208,1.006926
2,AB05 Albacete,0.915041,0.845700,33.454715,27.258544,1.037081
3,GC05 Arucas,0.662278,0.533650,17.160070,18.862297,0.666713
4,GC07 Tinajo,0.360861,0.746380,21.216336,25.682839,1.024135
5,IB04 Son Ferriol,0.898098,0.955711,47.240790,36.274414,1.103142
6,IB01 Santa Eulalia,0.895371,0.802939,39.407300,29.959426,0.940188
7,M03 Aranjuez,0.935287,1.412272,59.427244,57.364820,1.727579
8,V05 Cheste,0.891812,1.355943,63.549731,49.998118,1.480553
9,A19 Villena,0.911894,0.606177,24.305897,20.845936,0.751206


In [40]:
all_errors.describe()

,R2,MAE,MAPE,CVRMSE,RMSE
count,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.886706,0.823555,36.183863,31.158527,1.007922
std,0.119603,0.251680,12.908470,10.141532,0.281249
min,0.360861,0.425396,11.527883,13.830990,0.536653
25%,0.893591,0.659363,28.329250,25.636661,0.794387
50%,0.915041,0.802939,33.849189,29.342208,0.991303
75%,0.935822,0.927376,45.005603,36.239985,1.108101
max,0.963082,1.412272,63.549731,57.364820,1.727579
